In [1]:
!nvidia-smi

No devices were found


In [6]:
import torch
print(torch.backends.mps.is_available())

True


In [1]:
# !pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.4 MB/s eta 0:00:0000:0100:01


# Install YOLO

In [7]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()

# Train YOLOv8 Model on Custom Dataset

In [6]:
# !pip install roboflow

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="BjsTUwG8DMvVTG2rcnyz")
project = rf.workspace("mario-28fvy").project("mario-rl")
dataset = project.version(1).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.2, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Mario-RL-1 in yolov8:: 100%|██████████| 2106/2106 [00:00<00:00, 7632.54it/s]


In [ ]:
!yolo task=detect mode=train model='/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/runs/detect/train5/weights/last.pt' data={dataset.location}/data.yaml epochs=20 imgsz=640 device=mps

New https://pypi.org/project/ultralytics/8.1.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.2 🚀 Python-3.11.5 torch-2.1.2 MPS (Apple M3 Pro)
engine/trainer: task=detect, mode=train, model=/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/runs/detect/train5/weights/last.pt, data=/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/Mario-RL-1/data.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer

# Prediction (playground)

In [48]:
import yaml
import pandas as pd

with open('models/data.yaml','r') as file:
    data = yaml.safe_load(file)
    names = data['names']
    
    

In [9]:
# run this if you want to see bounding boxes on images
!yolo task=detect mode=predict model='/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/runs/detect/train6/weights/best.pt' conf=0.5 source='/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/Mario-RL-1/test/images/'

Ultralytics YOLOv8.1.2 🚀 Python-3.11.5 torch-2.1.2 CPU (Apple M3 Pro)
Model summary (fused): 218 layers, 25850761 parameters, 0 gradients, 78.7 GFLOPs

image 1/106 /Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/Mario-RL-1/test/images/SuperMarioBros-1-1-v0img_SuperMarioBros-1-1-v0_2024-01-19-15_17_22_png.rf.1fc272d9b63ffe303b20ec9aeb91c0a3.jpg: 640x640 1 brick, 1 ground, 1 mario, 2 pipes, 1 suprise_box, 189.2ms
image 2/106 /Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/Mario-RL-1/test/images/SuperMarioBros-1-1-v0img_SuperMarioBros-1-1-v0_2024-01-19-15_17_27_png.rf.9f575ceccdced4b4a72192f6ee654f7e.jpg: 640x640 2 goombas, 1 ground, 1 mario, 2 pipes, 180.8ms
image 3/106 /Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/Mario-RL-1/test/images/SuperMarioBros-1-1-v0img_SuperMarioBros-1-1-v0_2024-01-19-15_17_28_png.rf.70256f447f63660592dadc41295024e3.jpg: 640x640 1 ground, 1 mario, 177.6ms
image 4/106 /Users/maximvandecasteele/PycharmProjects/master

In [51]:
import pandas as pd

# return bounding boxes
model = YOLO('models/Mario_OD_vanilla_best.pt')


# just load an image or list of images into the model for prediction
# for result object   https://docs.ultralytics.com/modes/predict/#working-with-results
results = model('demo_frames/img_SuperMarioBros-1-1-v0_2024-01-17-16_08_35.png')

for r in results:
    boxes = r.boxes.numpy()
    classes = pd.DataFrame(boxes.cls, columns=['class'])
    xywh = pd.DataFrame(boxes.xywh, columns=['x', 'y', 'w', 'h'])
    positions = pd.concat([classes,xywh], axis=1)

positions = pd.DataFrame(data=None, columns=['name', 'class', 'x', 'y', 'w', 'h'])

for r in results:
            boxes = r.boxes.cpu().numpy()
            classes = pd.DataFrame(boxes.cls, columns=['class'])
            # other types of bounding box data can be chosen: xyxy, xywh, xyxyn, xywhn
            xywh = pd.DataFrame(boxes.xywh, columns=['x', 'y', 'w', 'h'])
            names_pd = classes['class'].apply(lambda x: names[int(x)])
            positions = pd.concat([classes, names_pd, xywh], axis=1)


image 1/1 /Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/demo_frames/img_SuperMarioBros-1-1-v0_2024-01-17-16_08_35.png: 608x640 4 brickss, 3 goombas, 2 surprise_boxs, 168.2ms
Speed: 1.3ms preprocess, 168.2ms inference, 0.5ms postprocess per image at shape (1, 3, 608, 640)
